In [2]:
import sqlite3
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import datetime

conn = sqlite3.connect("../goalflow.db")

# Example: aggregate weekly balance changes
query = """
SELECT 
    strftime('%Y-%W', datetime) AS week,
    SUM(CASE WHEN origin_account IS NOT NULL THEN -amount ELSE amount END) AS net_flow
FROM Transactions
GROUP BY week
ORDER BY week;
"""
df = pd.read_sql_query(query, conn)
conn.close()



In [3]:
df

,week,net_flow
0,2025-35,2420.0
1,2025-36,-950.0
2,2025-37,1000.0


In [4]:

# Add lag features (previous week flow)
df["prev_flow"] = df["net_flow"].shift(1)
df.dropna(inplace=True)

# Train a simple regression
X = df[["prev_flow"]]
y = df["net_flow"]

model = LinearRegression().fit(X, y)

# Forecast next week's flow
predicted = model.predict([[df["net_flow"].iloc[-1]]])[0]
print(f"Predicted next week's net flow: {predicted:.2f}")

Predicted next week's net flow: -128.34


c:\Users\ferlr\Code\hackmty\GoalFlow-Backend\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
